<a href="https://colab.research.google.com/github/demie20/Cascade_hackathon-/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display_html
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
seed=1
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
data_train= pd.read_csv('/content/train.csv')
data_train=shuffle(data_train,random_state=0)
ds_test= pd.read_csv('/content/test.csv')

In [ ]:
df=pd.DataFrame(data_train)
print(df.head())

          order_time  order_id  ... session_time cancelled_time
9929  1/26/2021 9:10  566682.0  ...   284.916667            NaN
6646  1/26/2021 8:07  563398.0  ...    72.233333            NaN
9423  1/26/2021 9:00  566176.0  ...   125.983333            NaN
7982  1/26/2021 8:33  564735.0  ...   173.550000            NaN
5205  1/26/2021 7:41  561958.0  ...   123.733333            NaN

[5 rows x 20 columns]


In [ ]:
train_labels=data_train["cancelled"]
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time','undelivered_orders',
                            'delivered_time','cancelled','order_id'],axis=1)

data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [ ]:
X = train_examples
y = train_labels

'''X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)'''

'X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)\nX_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)\nBATCH_SIZE = int((X_train.shape[0])/1000)'

In [ ]:
'''def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(3,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  opt = tf.keras.optimizers.SGD(lr=0.002)
  model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])         
  return model 
  '''

In [ ]:
'''from numpy.core.numeric import NaN
estimator = KerasRegressor(build_fn=get_basic_model, nb_epoch=100, batch_size=100, verbose=False)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator.fit(X, y)
prediction = estimator.predict(X)
#np.any(np.isnan(prediction))
#np.all(np.isfinite(prediction))
'''

Results: nan (nan) MSE


ValueError: ignored

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers


In [ ]:
regularizer_histories = {}
regularizer_histories['train'] = size_histories['train']

In [ ]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)
])
    model.compile(optimizer=opt,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])         
    return model 

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Dense(6, input_dim=6, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer="adam")
    return model


estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=100, verbose=False)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator.fit(X, y)
prediction = estimator.predict(X)
prediction=np.where(prediction > 0.5, 1, 0)

accuracy_score(y, prediction)

Results: -10.23 (18.32) MSE


0.9874755555555556

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y,prediction)

0.5044092034809413

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(all_labels, all_predications)

In [ ]:
np.isnan(y).any()

False

In [ ]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=4,verbose=1, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

Epoch 1/4
1058/1058 [==============================] - 3s 2ms/step - loss: nan - accuracy: 0.9416 - val_loss: nan - val_accuracy: 0.9906
Epoch 2/4
1058/1058 [==============================] - 3s 2ms/step - loss: nan - accuracy: 0.9884 - val_loss: nan - val_accuracy: 0.9906
Epoch 3/4
1058/1058 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9884 - val_loss: nan - val_accuracy: 0.9906
Epoch 4/4
1058/1058 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.9884 - val_loss: nan - val_accuracy: 0.9906


In [ ]:
model.evaluate(X_test,y_test)

1139/1139 [==============================] - 2s 2ms/step - loss: 0.0133 - accuracy: 0.9865


[0.013346037827432156, 0.9864979982376099]

In [ ]:
test=pd.read_csv("/content/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','undelivered_orders',
                            'order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)

In [ ]:
ans=model.predict(test)

In [ ]:
sum(ans)/len(ans)

array([nan])

In [ ]:
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)

In [ ]:
data.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [ ]:
data.to_csv("/content/model2.csv",index=False)